In [1]:
import sympy as sp
import sympy.physics.mechanics as me

import numpy as np

sp.init_printing()

In [9]:
# Forward kinematics
# Define the symbolic variables
xb, yb, pb, q = me.dynamicsymbols('xb yb pb q') # State
xbd, ybd, pbd, qd = me.dynamicsymbols('xb yb pb q', 1) # State derivatives
l, r = sp.symbols('l r') # Parameters

# Define the position equations
xe = xb + l*sp.sin(pb+q)
ye = yb - l*sp.cos(pb+q)
pe = pb + q

# Define the velocity equations
xe_dot = xbd + l*sp.cos(pb+q)*(pbd+qd)
ye_dot = ybd + l*sp.sin(pb+q)*(pbd+qd)
pe_dot = pbd + qd

# Rotation matrix of end effector in world frame
R_e = sp.Matrix([[sp.cos(pe), -sp.sin(pe)],
                 [sp.sin(pe),  sp.cos(pe)]])

# Geometric Jacobian in world frame
J = sp.Matrix([[1, 0, l*sp.cos(pb+q), l*sp.cos(pb+q)],
               [0, 1, l*sp.sin(pb+q), l*sp.sin(pb+q)],
               [0, 0, 1, 1]])

# Jacobian in end-effector frame
jacobian_rotation = sp.Matrix([[R_e, sp.zeros(2,1)],[sp.zeros(1,2), sp.eye(1)]])
J_e = jacobian_rotation * J

display(J_e)

⎡                                              2                      2        ↪
⎢cos(pb(t) + q(t))  -sin(pb(t) + q(t))  - l⋅sin (pb(t) + q(t)) + l⋅cos (pb(t)  ↪
⎢                                                                              ↪
⎢sin(pb(t) + q(t))  cos(pb(t) + q(t))      2⋅l⋅sin(pb(t) + q(t))⋅cos(pb(t) + q ↪
⎢                                                                              ↪
⎣        0                  0                                 1                ↪

↪                 2                      2              ⎤
↪ + q(t))  - l⋅sin (pb(t) + q(t)) + l⋅cos (pb(t) + q(t))⎥
↪                                                       ⎥
↪ (t))        2⋅l⋅sin(pb(t) + q(t))⋅cos(pb(t) + q(t))   ⎥
↪                                                       ⎥
↪                                1                      ⎦

In [10]:
# Lambdify the jacobian
# Turn into a function of the form x_dot = f(x, u)
x = sp.Matrix([xb, yb, pb, q])
p = sp.Matrix([l, r])
# Substitute all the time-dependent variables with symbols
J_e_subbed = J_e.subs({xb: sp.symbols('x_b', real=True, positive=True),
            yb: sp.symbols('y_b', real=True, positive=True),
            pb: sp.symbols('p_b', real=True, positive=True),
            q: sp.symbols('q', real=True, positive=True)})
display(J_e_subbed)
f = sp.lambdify((x, p), J_e_subbed, 'numpy')

⎡                                    2                 2                  2    ↪
⎢cos(p_b + q)  -sin(p_b + q)  - l⋅sin (p_b + q) + l⋅cos (p_b + q)  - l⋅sin (p_ ↪
⎢                                                                              ↪
⎢sin(p_b + q)  cos(p_b + q)      2⋅l⋅sin(p_b + q)⋅cos(p_b + q)        2⋅l⋅sin( ↪
⎢                                                                              ↪
⎣     0              0                         1                               ↪

↪               2         ⎤
↪ b + q) + l⋅cos (p_b + q)⎥
↪                         ⎥
↪ p_b + q)⋅cos(p_b + q)   ⎥
↪                         ⎥
↪       1                 ⎦

In [11]:
# Save the end-effector jacobian to file
model_name = '2d_simple_aerial_manipulator_end_effector_jacobian'
import dill
dill.settings['recurse'] = True
dill.dump(f, open(model_name+'.pkl', 'wb'))